I need take the rest.
I will upload the training part and add comment later to explain the detail.

Thanks for very great notebook. The notebook got many idea from the following.
1. https://www.kaggle.com/manikanthr5/riiid-sakt-model-training-public
2. https://www.kaggle.com/wangsg/a-self-attentive-model-for-knowledge-tracing
3. https://www.kaggle.com/leadbest/sakt-with-randomization-state-updates



In [1]:
import psutil
import joblib
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import gc
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('../input/practice-log-demographics/Practice_Log_Demographics.csv', low_memory=False)

In [3]:
Features =     [  'chapter_label', 'sub_chapter_label','user_id.x','term', 
                  'q', 'question_name', 'start_practice', 'end_practice'
                  ]

In [4]:
data = data[Features]

In [5]:
data['q'] = data['q'].apply(lambda a: a>=4)
data['q'] = data['q'].astype(np.int8)

In [6]:
gc.collect()

94

In [7]:
data.start_practice = pd.to_datetime(data.start_practice, format='%Y-%m-%d %H:%M:%S')
data.end_practice = pd.to_datetime(data.end_practice, format='%Y-%m-%d %H:%M:%S')
term_d = {'WN 2018': 0, 'FA 2018':1, 'WN 2019': 2, 'FA 2019':3, 'WN 2020': 4, 'FA 2020':5 }
data['chapter_id'] = data['chapter_label'].replace(data['chapter_label'].unique(), list(range(data['chapter_label'].nunique()))) + 1
data['sub_chapter_id'] = data['sub_chapter_label'].replace(data['sub_chapter_label'].unique(), list(range(data['sub_chapter_label'].nunique()))) + 1
data['question_id'] = data['question_name'].replace(data['question_name'].unique(), list(range(data['question_name'].nunique()))) + 1
data['term'] = data['term'].replace(term_d) + 1

In [8]:
data.drop(columns = ['chapter_label', 'sub_chapter_label', 'question_name'], inplace=True)

In [9]:
data = data.sort_values(['start_practice'], ascending=True).reset_index(drop=True)

In [10]:
data['user_id.x'] = data['term'].astype(str) + data['user_id.x'].astype(str)
data['user_id.x'] = data['user_id.x'].replace(data['user_id.x'].unique(), list(range(data['user_id.x'].nunique())))

In [11]:
data['prev_time_elapsed'] = None
data['time_lag'] = None
data['time_lag'] = data['time_lag'].astype(np.float)
data['prev_time_elapsed'] = data['prev_time_elapsed'].astype(np.float)

In [12]:
for user in tqdm(data['user_id.x'].unique()):
    tmp_user = data[data['user_id.x']==user]
    tmp_time_elapsed = tmp_user.end_practice.apply(lambda a: a.timestamp()) - tmp_user.start_practice.apply(lambda a: a.timestamp())
    #shifting time elapsed by one
    #so that time_elapsed row for each question
    #would refer to the time that user took to answer
    #previous question
    tmp_time_elapsed = np.insert(np.array(tmp_time_elapsed[:-1]), 0, -1., axis=0)
    indices = tmp_user.index
    start_row = indices[0]
    data['time_lag'].iloc[start_row] = -1
    time_substrahend = tmp_user.start_practice.iloc[:-1]
    time_substrahend = time_substrahend.apply(lambda a: a.timestamp())
    time_substrahend = np.array(time_substrahend)

    time_minuend = tmp_user.start_practice.iloc[1:]
    time_minuend = time_minuend.apply(lambda a: a.timestamp())
    time_minuend = np.array(time_minuend)
#     for indice in indices[1:]:
#         #time lag difference
#         data['time_lag'].iloc[indice] = data['start_practice'].iloc[indice] - data['start_practice'].iloc[start_row]
    data['prev_time_elapsed'].iloc[indices] = tmp_time_elapsed
    data['time_lag'].iloc[indices[1:]] = time_minuend - time_substrahend

100%|██████████| 887/887 [00:15<00:00, 56.46it/s]


In [13]:
#now we can drop end practice
data.drop(columns=['end_practice'], inplace=True)
data = data.sort_values(['start_practice'], ascending=True).reset_index(drop=True)
data['answered_correctly'] = data['q']
data.drop(columns='q', inplace=True)
gc.collect()

20

In [14]:
gc.collect()

20

In [15]:
## cv 0.7993

In [16]:
## epoch - 0 train_loss - 0.4103 train_auc - 0.7990 val_loss - 0.5121 val_auc - 0.7993 time=641.96s

In [17]:
MAX_SEQ = 500
n_part = data['chapter_id'].nunique() + 1
D_MODEL = 128
N_LAYER = 2
DROPOUT = 0.2

In [30]:
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(DROPOUT)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class SAINTModel(nn.Module):
    def __init__(self, n_skill, n_part, max_seq=MAX_SEQ, embed_dim= D_MODEL, elapsed_time_cat_flag = False):
        super(SAINTModel, self).__init__()

        self.n_skill = n_skill
        self.embed_dim = embed_dim
        self.n_cat = n_part
        self.elapsed_time_cat_flag = elapsed_time_cat_flag

        self.e_embedding = nn.Embedding(self.n_skill+1, embed_dim) ## exercise
        self.c_embedding = nn.Embedding(self.n_cat+1, embed_dim) ## category
        self.pos_embedding = nn.Embedding(max_seq+1, embed_dim) ## position
        self.res_embedding = nn.Embedding(2+1, embed_dim) ## response
        self.term_embedding = nn.Embedding(7, embed_dim)

        if self.elapsed_time_cat_flag == True:
            self.elapsed_time_embedding = nn.Embedding(300+1, embed_dim) ## elapsed time (the maximum elasped time is 300)
            self.lag_embedding1 = nn.Embedding(300+1, embed_dim) ## lag time1 for 300 seconds
            self.lag_embedding2 = nn.Embedding(1440+1, embed_dim) ## lag time2 for 1440 minutes
            self.lag_embedding3 = nn.Embedding(365+1, embed_dim) ## lag time3 for 365 days

        else:
            self.elapsed_time_embedding = nn.Linear(1, embed_dim, bias=False) ## elapsed time
            self.lag_embedding = nn.Linear(1, embed_dim, bias=False) ## lag time



        self.transformer = nn.Transformer(nhead=8, d_model = embed_dim, num_encoder_layers= N_LAYER, num_decoder_layers= N_LAYER, dropout = DROPOUT)

        self.dropout = nn.Dropout(DROPOUT)
        self.layer_normal = nn.LayerNorm(embed_dim) 
        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, question, part, response, elapsed_time, lag_time):

        device = question.device  
        ## embedding layer
        question = self.e_embedding(question)
        part = self.c_embedding(part)
        pos_id = torch.arange(question.size(1)).unsqueeze(0).to(device)
        pos_id = self.pos_embedding(pos_id)
        res = self.res_embedding(response)
        
        if self.elapsed_time_cat_flag == True:

            ## feature engineering
            ## elasped time
            elapsed_time = torch.true_divide(elapsed_time, 1000)
            elapsed_time = torch.round(elapsed_time)
            elapsed_time = torch.where(elapsed_time.float() <= 300, elapsed_time, torch.tensor(300.0).to(device)).long()
            elapsed_time = self.elapsed_time_embedding(elapsed_time)

            ## lag_time1
            lag_time = torch.true_divide(lag_time, 1000)
            lag_time = torch.round(lag_time)
            lag_time1 = torch.where(lag_time.float() <= 300, lag_time, torch.tensor(300.0).to(device)).long()

            ## lag_time2
            lag_time = torch.true_divide(lag_time, 60)
            lag_time = torch.round(lag_time)
            lag_time2 = torch.where(lag_time.float() <= 1440, lag_time, torch.tensor(1440.0).to(device)).long()

            ## lag_time3
            lag_time = torch.true_divide(lag_time, 1440)
            lag_time = torch.round(lag_time)
            lag_time3 = torch.where(lag_time.float() <= 365, lag_time, torch.tensor(365.0).to(device)).long()

            ## lag time
            lag_time1 = self.lag_embedding1(lag_time1) 
            lag_time2 = self.lag_embedding2(lag_time2) 
            lag_time3 = self.lag_embedding3(lag_time3)

        else:

            elapsed_time = elapsed_time.view(-1,1)
            elapsed_time = self.elapsed_time_embedding(elapsed_time)
            elapsed_time = elapsed_time.view(-1, MAX_SEQ, self.embed_dim)

            lag_time = lag_time.view(-1,1)
            lag_time = self.lag_embedding(lag_time)
            lag_time = lag_time.view(-1, MAX_SEQ, self.embed_dim)

            # elapsed_time = elapsed_time.view(-1, MAX_SEQ-1, 1)  ## [batch, s_len] => [batch, s_len, 1]
            # elapsed_time = self.elapsed_time_embedding(elapsed_time)


        enc = question + pos_id + part
        dec = pos_id + elapsed_time + lag_time + pos_id + res
        enc = enc.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        dec = dec.permute(1, 0, 2)
        mask = future_mask(enc.size(0)).to(device)
        att_output = self.transformer(enc, dec, src_mask=mask, tgt_mask=mask, memory_mask = mask)
        att_output = self.layer_normal(att_output)
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1)

## Load Pretrained Models

In [19]:
n_skill = data['question_id'].nunique() + 1

In [20]:
n_skill

1065

In [21]:
# questions_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')

In [22]:
# time_dict = joblib.load("../input/saint-plus-data-new/time_dict.pkl.zip")

In [23]:
model = SAINTModel(n_skill, n_part, embed_dim= D_MODEL)

# try:
#     model.load_state_dict(torch.load("../input/saint-plus-model/saint_plus_model_20210108_v1.pt"))
# except:
#     model.load_state_dict(torch.load("../input/saint-plus-model/saint_plus_model_20210108_v1.pt", map_location='cpu'))
# model.to(device)
# model.eval()

In [24]:
class PRACTICE_DATASET(Dataset):
    def __init__(self, data, maxlength=MAX_SEQ):
        super(PRACTICE_DATASET, self).__init__()
        self.maxlength = maxlength
        self.data = data
        self.test = test
        self.users = list()
        for user in data.index:
            self.users.append(user)
            
    def __len__(self):
        return len(self.users)
    
    def __getitem__(self, ix):
        user = self.users[ix]
        question_id, parts, pri_elap, lag, labels = self.data[user]
        question_id = np.array(question_id, np.int16)
        parts = np.array(parts, np.int16)
        pri_elap = np.array(pri_elap, np.float)
        lag = np.array(lag, np.float)
        labels = np.array(labels, np.int8)
        n = len(question_id)
        if n > self.maxlength:
            question_id = question_id[-self.maxlength : ]
            parts = parts[-self.maxlength :]
            pri_elap = pri_elap[-self.maxlength :]
            labels = labels[-self.maxlength: ]
            lag = lag[-self.maxlength: ]
            responses = np.append(2, labels[:-1])
        else:
            question_id = np.pad(question_id, (self.maxlength - n, 0))
            parts = np.pad(parts, (self.maxlength - n, 0))
            pri_elap = np.pad(pri_elap, (self.maxlength - n, 0))
            lag = np.pad(lag, (self.maxlength - n, 0))
            responses = np.append(2, labels[:-1])
            labels = np.pad(labels, (self.maxlength - n, 0))
            responses = np.pad(responses, (self.maxlength - n, 0), mode='constant', constant_values = 2)
        return question_id, parts, responses, pri_elap, lag, labels 

In [25]:
group = data[['user_id.x', 'question_id', 'answered_correctly', 'sub_chapter_id',
                          'chapter_id', 'prev_time_elapsed', 'time_lag', 'term']].groupby('user_id.x').apply(lambda r: (
                                                                                                r['question_id'].values,
                                                                                                r['chapter_id'].values,
                                                                                                r['prev_time_elapsed'].values,
                                                                                                r['time_lag'].values,
                                                                                                r['answered_correctly'].values,
))

In [26]:
patience = 5

In [31]:
X = np.array(group.keys())
kfold = KFold(n_splits=5, shuffle=True)
train_losses = list()
train_aucs = list()
train_accs = list()
val_losses = list()
val_aucs = list()
val_accs = list()
test_losses = list()
test_aucs = list()
test_accs = list()
for train, test in kfold.split(X):
    users_train, users_test =  X[train], X[test]
    n = len(users_test)//2
    users_test, users_val = users_test[:n], users_test[n: ]
    train = PRACTICE_DATASET(group[users_train])
    valid = PRACTICE_DATASET(group[users_val])
    test = PRACTICE_DATASET(group[users_test])
    train_dataloader = DataLoader(train, batch_size=64, shuffle=True, num_workers=8)
    val_dataloader = DataLoader(valid, batch_size=64, shuffle=True, num_workers=8)
    test_dataloader = DataLoader(test, batch_size=64, shuffle=True, num_workers=8)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    saint = SAINTModel(n_skill, n_part)
    epochs = 100
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(saint.parameters(), betas=(0.9, 0.999), lr = 0.0005, eps=1e-8)
    saint.to(device)
    criterion.to(device)
    
    def train_epoch(model=saint, train_iterator=train_dataloader, optim=optimizer, criterion=criterion, device=device):
        model.train()

        train_loss = []
        num_corrects = 0
        num_total = 0
        labels = []
        outs = []
        tbar = tqdm(train_iterator)
        for item in tbar:
            question_id = item[0].to(device).long()
            parts = item[1].to(device).long()
            responses = item[2].to(device).long()
            pri_elap = item[3].to(device).float()
            lag = item[4].to(device).float()
            label = item[5].to(device).float()
            target_mask = (question_id!=0)
            optim.zero_grad()
            output = model(question_id, parts, responses, pri_elap, lag)
            output = torch.reshape(output, label.shape)

            output = torch.masked_select(output, target_mask)
            label = torch.masked_select(label, target_mask)

            loss = criterion(output, label)
            loss.backward()
            optim.step()
            train_loss.append(loss.item())
            pred = (torch.sigmoid(output) >= 0.5).long()

            num_corrects += (pred == label).sum().item()
            num_total += len(label)

            labels.extend(label.view(-1).data.cpu().numpy())
            outs.extend(output.view(-1).data.cpu().numpy())

            tbar.set_description('loss - {:.4f}'.format(loss))
        acc = num_corrects / num_total
        auc = roc_auc_score(labels, outs)
        loss = np.mean(train_loss)

        return loss, acc, auc
   

    def val_epoch(model=saint, val_iterator=test_dataloader, 
              criterion=criterion, device=device):
        model.eval()

        train_loss = []
        num_corrects = 0
        num_total = 0
        labels = []
        outs = []
        tbar = tqdm(val_iterator)
        for item in tbar:
            question_id = item[0].to(device).long()
            parts = item[1].to(device).long()
            responses = item[2].to(device).long()
            pri_elap = item[3].to(device).float()
            lag = item[4].to(device).float()
            label = item[5].to(device).float()
            target_mask = (question_id!=0)
            with torch.no_grad():
                output = model(question_id, parts, responses, pri_elap, lag)

            output = torch.reshape(output, label.shape)
            output = torch.masked_select(output, target_mask)
            label = torch.masked_select(label, target_mask)

            loss = criterion(output, label)
            train_loss.append(loss.item())

            pred = (torch.sigmoid(output) >= 0.5).long()
            num_corrects += (pred == label).sum().item()
            num_total += len(label)

            labels.extend(label.view(-1).data.cpu().numpy())
            outs.extend(output.view(-1).data.cpu().numpy())

            tbar.set_description('valid loss - {:.4f}'.format(loss))

        acc = num_corrects / num_total
        auc = roc_auc_score(labels, outs)
        loss = np.average(train_loss)

        return loss, acc, auc
    
    MIN_VAL = 1000000000
    count = 0
    print('----------------------------------------------------------------------------')
    for epoch in range(epochs):
        train_loss, train_acc, train_auc = train_epoch(model=saint, device=device)
        print("epoch - {} train_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, train_loss, train_acc, train_auc))
        val_loss, val_acc, val_auc = val_epoch(model=saint, val_iterator= val_dataloader, device=device)
        print("epoch - {} val_loss - {:.2f} val acc - {:.3f} val auc - {:.3f}".format(epoch, val_loss, val_acc, val_auc))
        if val_loss < MIN_VAL:
            count = 0
            MIN_VAL = val_loss
        else:
            count += 1
        if count == patience:
            print('Val Loss does not improve for {} consecutive epochs'.format(patience))
            break
    test_loss, test_acc, test_auc = val_epoch(model=saint, device=device)
    print("epoch - {} test_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, test_loss, test_acc, test_auc))
    train_loss, train_acc, train_auc = val_epoch(model=saint, val_iterator=train_dataloader, device=device)

    test_losses.append(test_loss)
    test_aucs.append(test_auc)
    test_accs.append(test_acc)
    train_aucs.append(train_auc)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

  0%|          | 0/12 [00:00<?, ?it/s]

----------------------------------------------------------------------------


  0%|          | 0/2 [00:00<?, ?it/s]

epoch - 0 train_loss - 0.84 acc - 0.502 auc - 0.510


  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

epoch - 15 val_loss - 0.66 val acc - 0.605 val auc - 0.646
Val Loss does not improve for 5 consecutive epochs


  0%|          | 0/12 [00:00<?, ?it/s]

epoch - 15 test_loss - 0.66 acc - 0.629 auc - 0.669


valid loss - 0.6069: 100%|██████████| 12/12 [00:01<00:00,  6.13it/s]


In [32]:
print("test avg loss: ", np.mean(test_losses), np.std(test_losses))
print("test avg acc: ", np.mean(test_accs), np.std(test_accs))
print("test avg auc: ", np.mean(test_aucs), np.std(test_aucs))

test avg loss:  0.653725516796112 0.0033334274563024133
test avg acc:  0.62575352297477 0.005844137525462814
test avg auc:  0.6691695828256311 0.010050423371214066


In [34]:
print("train avg loss: ", np.mean(train_losses), np.std(train_losses))
print("train avg acc: ", np.mean(train_accs), np.std(train_accs))
print("train avg auc: ", np.mean(train_aucs), np.std(train_aucs))

train avg loss:  0.6349065115054449 0.0066331861774546344
train avg acc:  0.6378740125253036 0.006030156587145543
train avg auc:  0.6876014883335031 0.00493965601919702
